<font size=6>Успеваемость студентов НИТУ МИСИС</font>
<br>

Cтудент во время учебы должен получить максимальное количество полезных знаний, чтобы в дальнейшем стать специалистом в своем направление. Задачей института является предоставление этих знаний.

Студенты часто теряют контроль над своей успеваемостью и "тонут" в пересдачах. Отдел аналитики способен контролировать обучение студента, анализируя его историю. На основе аналитики мы можем дать оценку: насколько близок студент к отчислению и насколько упал уровень успеваемости. А также каким предметам стоит уделить должное внимание.

<b>Задача</b> состоит в разработке модели, способной предсказывать уровень успеваемости студента.

<br>
<br>
<b>Данные для исследования:</b>

hash - уникальный номер студента
Номер ЛД - доп. идентификация студента
Уровень подготовки - (Академ Бакалавр = Бакалавр, Специалист, Магистр, Аспирант)
Учебная группа - сокращенное название (первая буквы = уровень подготовки)
Специальность/направление
Учебный год
Полугодие
Дисциплина
Оценка (без пересдач) - промежуточная оценка с возможностью пересдачи
Оценка (успеваемости) - итоговая оценка за предмет

In [126]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

import plotly.express as px

<font size=5>Этап 1. Изучение данных EDA</font>

In [127]:
data = pd.read_excel('data/students.xlsx')
data

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
0,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Инженерная компьютерная графика,Хорошо,Хорошо
1,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Иностранный язык,Удовлетворительно,Удовлетворительно
2,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,История,зачтено,зачтено
3,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Математика,Хорошо,Хорошо
4,67a80fffd8d0294a596eda117d7e393c,2010218,Академический бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Программирование и алгоритмизация,Хорошо,Хорошо
...,...,...,...,...,...,...,...,...,...,...
376002,53f3767de3a82def8de1e36aba70dd95,2302623,Бакалавр,БЭК-23-6,"Экономика и управление (Экономика,Менеджмент)",2023 - 2024,II полугодие,Математика,NaN,NaN
376003,53f3767de3a82def8de1e36aba70dd95,2302623,Бакалавр,БЭК-23-6,"Экономика и управление (Экономика,Менеджмент)",2023 - 2024,II полугодие,Право,NaN,NaN
376004,53f3767de3a82def8de1e36aba70dd95,2302623,Бакалавр,БЭК-23-6,"Экономика и управление (Экономика,Менеджмент)",2023 - 2024,II полугодие,Теория отраслевых рынков,NaN,NaN
376005,53f3767de3a82def8de1e36aba70dd95,2302623,Бакалавр,БЭК-23-6,"Экономика и управление (Экономика,Менеджмент)",2023 - 2024,II полугодие,Физическая культура и спорт,NaN,NaN


In [128]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376007 entries, 0 to 376006
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   hash                       376007 non-null  object
 1   Номер ЛД                   376007 non-null  int64 
 2   Уровень подготовки         376007 non-null  object
 3   Учебная группа             376007 non-null  object
 4   Специальность/направление  376007 non-null  object
 5   Учебный год                376007 non-null  object
 6   Полугодие                  376007 non-null  object
 7   Дисциплина                 376007 non-null  object
 8   Оценка (без пересдач)      189004 non-null  object
 9   Оценка (успеваемость)      196522 non-null  object
dtypes: int64(1), object(9)
memory usage: 28.7+ MB


Проверим, что признак <b>Номер ЛД</b> также идентифицирует студента

In [129]:
data['hash'].nunique() == data['Номер ЛД'].nunique()

False

In [130]:
data.groupby('hash')['Номер ЛД'].nunique().sort_values(ascending=False)

hash
2f41ab589489046803d432b59e8e6331    2
c38a2fd37e008e520c954bef41b98d65    2
bc2b18ddce3cb9fe20fb47af8d0390cf    2
5eb84149f57391b4d71b55703e3bf496    2
ad5b37dae95b7b05b6fcebaa98a69490    2
                                   ..
54f412831540c9d44ed91b6a20454927    1
54ebb737e02fcadc1a3aba0117a42da0    1
54eb43c1ae635bca0be1b2c8ddde3192    1
54dca6304823a20fcc7440579e02f38e    1
ffed4fe2ad8cbff97e0b3d21da23008d    1
Name: Номер ЛД, Length: 9085, dtype: int64

In [131]:
data[data['hash'] == '2f41ab589489046803d432b59e8e6331'].iloc[65:70]

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
275277,2f41ab589489046803d432b59e8e6331,2005639,Академический бакалавр,БМТМ-20-2,Материаловедение и технологии материалов,2023 - 2024,II полугодие,Подготовка к процедуре защиты и защита выпускн...,NaN,NaN
275278,2f41ab589489046803d432b59e8e6331,2005639,Академический бакалавр,БМТМ-20-2,Материаловедение и технологии материалов,2023 - 2024,II полугодие,Преддипломная практика для выполнения выпускно...,NaN,NaN
275279,2f41ab589489046803d432b59e8e6331,2005639,Академический бакалавр,БМТМ-20-2,Материаловедение и технологии материалов,2023 - 2024,II полугодие,Структура и свойства металлических наноматериалов,NaN,NaN
275280,2f41ab589489046803d432b59e8e6331,2205842,Бакалавр,БМТМ-22-2,Материаловедение и технологии материалов,2022 - 2023,I полугодие,Инженерная и компьютерная графика,зачтено,зачтено
275281,2f41ab589489046803d432b59e8e6331,2205842,Бакалавр,БМТМ-22-2,Материаловедение и технологии материалов,2022 - 2023,I полугодие,Иностранный язык,Отлично,Отлично


Некоторые студенты имеют одинаковый hash. Скорей всего, это ФИО.
Поэтому идентифицировать студентов будем по признаку <b>Номер ЛД</b>

Изучим остальные признаки подробней

In [132]:
# По условию Академический бакалавр == Бакалавр
data['Уровень подготовки'] = data['Уровень подготовки'].replace('Академический бакалавр', 'Бакалавр')

year_counts = (data
               .groupby(['Номер ЛД'])['Уровень подготовки']
               .value_counts()
               .reset_index()
               .loc[:, 'Уровень подготовки']
               .value_counts()
               .reset_index())

fig = px.bar(year_counts, x='Уровень подготовки', y='count')
fig.show()

In [133]:
data['Учебный год'].value_counts()

Учебный год
2023 - 2024    158114
2022 - 2023    113950
2021 - 2022     67558
2020 - 2021     28226
2019 - 2020      5520
2018 - 2019      2299
2024 - 2025       282
2025 - 2026        37
2017 - 2018        15
2026 - 2027         5
2027 - 2028         1
Name: count, dtype: int64

Удалим данные за будущие года

In [134]:
future_list = ['2024 - 2025', '2025 - 2026', '2026 - 2027', '2027 - 2028']
future_index = data[data['Учебный год'].isin(future_list)].index
data = data.drop(future_index)
data['Учебный год'].value_counts()

Учебный год
2023 - 2024    158114
2022 - 2023    113950
2021 - 2022     67558
2020 - 2021     28226
2019 - 2020      5520
2018 - 2019      2299
2017 - 2018        15
Name: count, dtype: int64

In [135]:
data['Полугодие'].value_counts()

Полугодие
II полугодие    195684
I полугодие     179998
Name: count, dtype: int64

In [136]:
data['Оценка (без пересдач)'].value_counts()

Оценка (без пересдач)
Отлично                 54500
Хорошо                  37402
зачтено                 34523
Удовлетворительно       28440
Неявка                  16339
Неудовлетворительно     12504
не зачтено               4840
Неявка по ув.причине      345
Не допущен                 13
Name: count, dtype: int64

In [137]:
data['Оценка (успеваемость)'].value_counts()

Оценка (успеваемость)
Отлично                 57818
Удовлетворительно       44514
зачтено                 42729
Хорошо                  42694
Неявка                   5504
не зачтено               1576
Неудовлетворительно      1419
Неявка по ув.причине        9
Не допущен                  1
Name: count, dtype: int64

<font size=5>Этап 2. Обработка данных</font>

Переведем оценки в числовой вид.
Каждая оценка будет означать балл, который получил студент за экзамен.
В дальнейшем будем использовать эти баллы для определения индекса успеваемости студента.

Зачет будет оцениваться в 1 балл, если он прошел успешно.
Если студент не сдал экзамен - 0 баллов (сюда входит и отсутствие на экзамене)

In [138]:
grade_to_points = {
    "Отлично": 5,
    "Хорошо": 4,
    "Удовлетворительно": 3,
    "Неудовлетворительно": 0,
    "зачтено": 1,
    "не зачтено": 0,
    "Неявка": 0,
    "Неявка по ув.причине": 0,
    "Не допущен": 0
}


def exam_type(df: pd.DataFrame):
    return df['Оценка (успеваемость)'].map(lambda x: 'зачет' if x in ['зачтено', 'не зачтено'] else 'экзамен')
    
grade_data = (data
              .assign(grades=lambda df: df['Оценка (успеваемость)'].map(grade_to_points),
                      type=exam_type)
              .drop(['Оценка (без пересдач)', 'Оценка (успеваемость)'], axis=1))
grade_data.head()

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,grades,type
0,67a80fffd8d0294a596eda117d7e393c,2010218,Бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Инженерная компьютерная графика,4.0,экзамен
1,67a80fffd8d0294a596eda117d7e393c,2010218,Бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Иностранный язык,3.0,экзамен
2,67a80fffd8d0294a596eda117d7e393c,2010218,Бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,История,1.0,зачет
3,67a80fffd8d0294a596eda117d7e393c,2010218,Бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Математика,4.0,экзамен
4,67a80fffd8d0294a596eda117d7e393c,2010218,Бакалавр,БИВТ-20-4,Информатика и вычислительная техника,2020 - 2021,I полугодие,Программирование и алгоритмизация,4.0,экзамен


Во время анализа появилась проблема с интерпретацией пересдач.
У некоторых студентов история идеально подходит под понятие пересдачи.
Как например 70272 - студент получил неудовлетворительно по математике, а после пересдал. Отсюда можно легко собрать показатель пересдач - сколько раз предмет повторился в семестре (полугодие), а окончательная оценка - наилучшая (в пример 3).

In [139]:
data[data['Номер ЛД'] == 70272].iloc[:9, 5:]

,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
156556,2019 - 2020,II полугодие,Иностранный язык,NaN,NaN
156557,2019 - 2020,II полугодие,Информатика,NaN,NaN
156558,2019 - 2020,II полугодие,История,NaN,зачтено
156559,2019 - 2020,II полугодие,История экономических учений,NaN,Удовлетворительно
156560,2019 - 2020,II полугодие,Математика,NaN,NaN
156561,2019 - 2020,II полугодие,Математика,NaN,Удовлетворительно
156562,2019 - 2020,II полугодие,Микроэкономика,NaN,Удовлетворительно
156563,2019 - 2020,II полугодие,Персональная эффективность,NaN,зачтено
156564,2019 - 2020,II полугодие,Философия,NaN,NaN


Но есть другой пример - 2112656.
Где результаты словно дублируются.

In [140]:
data[data['Номер ЛД'] == 2112656].head(6)

,hash,Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,Учебный год,Полугодие,Дисциплина,Оценка (без пересдач),Оценка (успеваемость)
375475,1179a1580768998448c8b2e07236ddea,2112656,Магистр,МЭЭ-22-1,Электроэнергетика и электротехника,2022 - 2023,I полугодие,Дополнительные главы математики,зачтено,зачтено
375476,1179a1580768998448c8b2e07236ddea,2112656,Магистр,МЭЭ-22-1,Электроэнергетика и электротехника,2022 - 2023,I полугодие,Дополнительные главы математики,не зачтено,зачтено
375477,1179a1580768998448c8b2e07236ddea,2112656,Магистр,МЭЭ-22-1,Электроэнергетика и электротехника,2022 - 2023,I полугодие,Иностранный язык,зачтено,зачтено
375478,1179a1580768998448c8b2e07236ddea,2112656,Магистр,МЭЭ-22-1,Электроэнергетика и электротехника,2022 - 2023,I полугодие,Иностранный язык,не зачтено,зачтено
375479,1179a1580768998448c8b2e07236ddea,2112656,Магистр,МЭЭ-22-1,Электроэнергетика и электротехника,2022 - 2023,I полугодие,Производственная практика,Неявка,Отлично
375480,1179a1580768998448c8b2e07236ddea,2112656,Магистр,МЭЭ-22-1,Электроэнергетика и электротехника,2022 - 2023,I полугодие,Производственная практика,Отлично,Отлично


Все же попробуем интерпретировать повторение дисциплин в одном семестре, как пересдачу, в иных случаях было бы странно делать дубликат результатов.

По условию задачи нужно убрать у каждого студента 20% пропусков.

Сгруппируем студентов по экзаменам и найдем максимальную оценку, если ее нет, то по экзамену будет NaN). 
Также найдем сколько раз каждый предмет встретился в семестре.

In [141]:
%%time
np.random.seed(42)


grades_data = (grade_data
               .groupby(['Номер ЛД', 'Уровень подготовки', 'Учебная группа',
                         'Специальность/направление', 'Учебный год', 
                         'Полугодие', 'type', 'Дисциплина'])
               .agg(repeats=('grades', 'size'),
                    grades=('grades', 'max'))
               .reset_index())


def missing_delete(students: pd.DataFrame, delete_percent: float) -> pd.DataFrame:
    """
    Функция для удаления определенного процента пропусков.
    Цикл перебирает каждого студента, находит у него null 
    и определяет какое количество необходимо удалить.
    С помощью рандома выбираются индексы для удаления
    и записываются в общий список индексов.
    :param students: Датасет студентов;
    :param delete_percent: Процент удаленных null;
    :return: Датасет студентов с удаленным процентом пропусков
    """
    students_data = students.copy()
    all_index_drop = []
    
    for student_id in students_data['Номер ЛД'].unique():
        current_student = students_data[students_data['Номер ЛД'] == student_id]
        
        missing_grades = current_student[current_student['grades'].isnull()]
        subject_num = missing_grades.shape[0]
        delete_num = round(subject_num * delete_percent)
        
        random_indexes = np.random.choice(missing_grades.index, delete_num)
        
        all_index_drop.extend(random_indexes)
    
    cleaned_grades_data = (students_data
                           .drop(all_index_drop))
    
    return cleaned_grades_data

clear_data = missing_delete(students=grades_data, delete_percent=0.2)

CPU times: user 2.51 s, sys: 80.7 ms, total: 2.59 s
Wall time: 2.69 s


Разделим датасет на отдельные периоды.
Пропуски в прошлых семестрах заменил на балл 0 (неудовлетворительно)

In [142]:
current_year = clear_data.query('`Учебный год` == "2023 - 2024"')
previous_year = clear_data.query('`Учебный год` != "2023 - 2024"').fillna(0)

In [143]:
test_current_year = current_year.query('Полугодие == "I полугодие"').fillna(0)
predict_current_year = current_year.query('Полугодие == "II полугодие"')

Теперь необходимо собрать статистику каждого студента по семестрам.
Моя идея заключается в определении "индекса успеваемости", который способен оценить уровень студента по его экзаменам.

Индекс будем находить по формуле:


$$\displaystyle{{performance\, exam\, index = \frac{кол\!-\!во\; эказменов}{кол\!-\!во\; предметов} * \frac{набранные\; баллы(экзамены)}{максимальные\; баллы\; (зачеты)}}}$$

$$\displaystyle{{максимальные\; баллы\; (экзамен) = кол\!-\!во\; экзаменов * 5}}$$

$$\displaystyle{{performance\, test\, index = \frac{кол\!-\!во\; зачетов}{кол\!-\!во\; предметов} * \frac{набранные\; баллы(зачеты)}{максимальные\; баллы\; (зачеты)}}}$$

$$\displaystyle{{максимальные\; баллы\; (зачеты) = кол\!-\!во\; зачетов * 1}}$$

<br>
<br>

$$\displaystyle{{performance\, index = performance\, exam\, index + performance\, test\, index}}$$

Таким образом, мы можем оценить успеваемость студента от [0, 1]

Теперь необходимо обработать датасет, чтобы увидеть историю семестров с точки зрения основных метрик: кол-во пересдач, общее кол-во предметов, индекс успеваемости.

Для более удобной работы с периодами обучения переведем их в соответствующий тип данных.
Для этого 1 полугодие переведем в 1 декабря, как зимнюю сессию, а 2 полугодие в 1 июня, как летнюю сессию.

In [144]:
def count_performance(df: pd.DataFrame) -> pd.Series:
    """
    Функция для нахождения коэффициента экзаменов и коэффициента зачетов.
    :param df: Датасет со студентами;
    :return: Обработанный датасет с индексами экзаменов и коэффициентов
    """    
    subject_coeff = df['subject_unique'] / df['total_subjects']
    
    max_points = np.where(df['type'] == 'зачет', 1, 5)
    grade_coeff = df['total_points'] / (df['subject_unique'] * max_points)
    
    performance_coef = subject_coeff * grade_coeff
    
    return performance_coef


def metrics_counter(df: pd.DataFrame) -> pd.DataFrame:
    """
    Функция для нахождения основных метрик семестра:
    кол-во пересдач, кол-во экзаменов/зачетов, а также
    общее кол-во предметов в семестре.
    :param df: Датасет со студентами;
    :return: Обработанный датасет с основными метриками.
    """
    subject_counter = (df
                       .groupby(['Номер ЛД', 'Уровень подготовки',
                                 'Учебная группа', 'Специальность/направление', 'period'])
                       .agg(total_subjects=('Дисциплина', 'count')))
    
    metrics_counter =  (df
                        .groupby(['Номер ЛД', 'Уровень подготовки', 'Учебная группа', 
                                  'Специальность/направление', 'period', 'type'])
                        .agg(total_points=('grades', 'sum'),
                             subject_unique=('Дисциплина', 'nunique'), 
                             subject_repeat=('repeats', 'sum')))
    
    return pd.merge(metrics_counter, subject_counter, 
                     how='outer', left_index=True, right_index=True).reset_index() 


def convert_period(period_str: str) -> pd.Timestamp:
    """
    Функция для конвертации периода.
    Переводит год и полугодие в формате Timestamp.
    1 полугодие - зимний семестр, поэтому дата будет 1 декабря
    2 полугодие - летний семестр, поэтому дата будет 1 июня
    :param period_str: период в формате 'year_start - year_end num_semester'
    :return: Timestamp в формате год-12-01 или год-06-01
    """
    period_list = period_str.split()
    
    year_part = ''.join(period_list[:3])
    year_start, year_end = year_part.split('-')

    semester_part = ''.join(period_list[3])
    
    if semester_part == 'I':
        return pd.Timestamp(f'{year_start}-12-01') # Зимняя сессия
    elif semester_part == 'II':
        return pd.Timestamp(f'{year_end}-06-01') # Летняя сессия 
    
    
def history_processing(data: pd.DataFrame) -> pd.DataFrame:
    """
    Функция для обработки истории студента.
    Ее задача найти метрики семестра: кол-во пересдач, 
    кол-во предметов и индекс успеваемости
    :param data: Датасет со студентами;
    :return: Обработанный датасет, где каждая строка 
    показывает как студент сдал семестр.
    """
    return (data
            .assign(period=lambda df: (df['Учебный год'] + ' ' + df['Полугодие']).map(convert_period))
            .pipe(metrics_counter)
            .assign(repeat_num=lambda df: df['subject_repeat'] - df['subject_unique'],
                    type_performance_index=count_performance)
            .groupby(['Номер ЛД', 'Уровень подготовки', 'Учебная группа', 'Специальность/направление', 'period'])
            .agg(performance_index=('type_performance_index', 'sum'), repeat_num=('repeat_num', 'sum'),
                 subject_num=('subject_unique', 'sum')))
       
    
fprevious_year = history_processing(data=previous_year)
fprevious_year.head()

performance_index  \
Номер ЛД Уровень подготовки Учебная группа Специальность/направление period                          
70272    Бакалавр           БЭК-19-3з      Экономика                 2020-06-01           0.475000   
                                                                     2021-06-01           0.716667   
                                                                     2022-06-01           0.581818   
                                                                     2023-06-01           0.537500   
100255   Бакалавр           БМН-20-1з      Менеджмент                2020-12-01           0.760000   

                                                                                 repeat_num  \
Номер ЛД Уровень подготовки Учебная группа Специальность/направление period                   
70272    Бакалавр           БЭК-19-3з      Экономика                 2020-06-01           1   
                                                                     2021-06-01           1   
                                                                     2022-06-01           1   
                                                                     2023-06-01           3   
100255   Бакалавр           БМН-20-1з      Менеджмент                2020-12-01           0   

                                                                                 subject_num  
Номер ЛД Уровень подготовки Учебная группа Специальность/направление period                   
70272    Бакалавр           БЭК-19-3з      Экономика                 2020-06-01            8  
                                                                     2021-06-01           12  
                                                                     2022-06-01           11  
                                                                     2023-06-01           16  
100255   Бакалавр           БМН-20-1з      Менеджмент                2020-12-01            5

In [145]:
ftest_current_year = history_processing(data=test_current_year)
ftest_current_year.head()

,,,,,performance_index,repeat_num,subject_num
Номер ЛД,Уровень подготовки,Учебная группа,Специальность/направление,period,,,
100255,Бакалавр,БМН-20-1з,Менеджмент,2023-12-01,0.800000,0,8
122905,Бакалавр,БМТ-21-1,Металлургия,2023-12-01,0.400000,0,11
125428,Специалист,СГД-18-2,Горное дело,2023-12-01,0.733333,0,6
125738,Специалист,СГД-18-2,Горное дело,2023-12-01,0.600000,1,6
130264,Бакалавр,БМН-19-1з,Менеджмент,2023-12-01,0.688889,1,9


Напишем функцию для отображения динамики индекса успеваемости

In [146]:
import plotly.graph_objects as go


def performance_plot(sample_students: pd.DataFrame, 
                     max_students: int, predicted: bool = False) -> None:
    """
    Функция для отображения динамики индекса успеваемости.
    Она способна отображать динамику фактическую и предсказанную.
    :param sample_students: Датасет со студентов;
    :param max_students: Число студентов для отображения на графике;
    :param predicted: Есть предсказанные значения в датасете или нет.
    :return: None
    """
    fig = go.Figure()

    colors = [
        '#AEC6CF', 
        '#FFB347',  
        '#77DD77',  
        '#FF6961',
        '#CFCFC4', 
        '#FDFD96', 
        '#836953', 
        '#779ECB',  
        '#DEA5A4',  
        '#B39EB5'
    ]

    student_ids = sample_students['Номер ЛД'].unique()[:max_students]

    for i, student_id in enumerate(student_ids):
        student_data = sample_students[sample_students['Номер ЛД'] == student_id]

        color = colors[i % len(colors)]

        if predicted is True:
            real_data = student_data[student_data['real'] == True]
            predicted_data = student_data[student_data['real'] == False]
            if not real_data.empty:
                fig.add_trace(go.Scatter(
                    x=real_data['period'],
                    y=real_data['performance_index'],
                    mode='lines+markers',
                    name=f'Student {student_id} (Real)',
                    line=dict(color=color)
                ))

            if not predicted_data.empty:
                fig.add_trace(go.Scatter(
                    x=predicted_data['period'],
                    y=predicted_data['performance_index'],
                    mode='markers',
                    name=f'Student {student_id} (Predicted)',
                    marker=dict(color='purple', symbol='circle', size=10)
                ))

                if not real_data.empty:
                    fig.add_trace(go.Scatter(
                        x=[real_data['period'].iloc[-1], predicted_data['period'].iloc[0]],
                        y=[real_data['performance_index'].iloc[-1], predicted_data['performance_index'].iloc[0]],
                        mode='lines',
                        line=dict(color=color, dash='dash'),
                        showlegend=False
                    ))

        else:
            fig.add_trace(go.Scatter(
                x=student_data['period'],
                y=student_data['performance_index'],
                mode='lines+markers',
                name=f'Student {student_id}',
                line=dict(color=color)
            ))

    fig.update_layout(
        title='Индекс успеваемости для выбранных студентов',
        xaxis_title='Semester',
        yaxis_title='Performance Index',
        xaxis=dict(tickangle=-45),
        legend_title='Students',
    )

    fig.show()

In [147]:
rfprevious_year = fprevious_year.reset_index()
performance_plot(sample_students=rfprevious_year, max_students=3)

<font size=5>Этап 3. Создание и Обучение модели</font>

Первое о чем я подумал - сделать предсказание на основе временного ряда.
То есть взять период семестров, индекс успеваемости и делать предсказания на следующий семестр.

<font size=4>Предсказание индекса успеваемости на основе временного ряда. sktime</font>

In [148]:
from sktime.split import temporal_train_test_split
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.base import ForecastingHorizon
from sklearn.ensemble import RandomForestRegressor
from sktime.performance_metrics.forecasting import mean_absolute_error


def time_prediction(student_data: pd.DataFrame, 
                    predict_date: pd.Timestamp = None,
                    wl: int = 3) -> tuple[float, pd.Series]:
    student_data.set_index('period', inplace=True)
    student_data.index = student_data.index.to_period('6M')
    
    y_train, y_test = temporal_train_test_split(student_data.loc[:, 'performance_index'], test_size=0.2)
    if predict_date is not None:
        fh = ForecastingHorizon(predict_date, is_relative=False)
    else:
        fh = ForecastingHorizon(y_test.index, is_relative=False)
        
    regressor = RandomForestRegressor()
    model = make_reduction(regressor, window_length=wl, strategy='recursive')
    model.fit(y_train)
    predict = model.predict(fh)

    mae = mean_absolute_error(predict, y_test)
    return mae, predict

In [149]:
%%time
error_list = []
for student_id in rfprevious_year['Номер ЛД'].unique()[:1000]:
    try:
        student_data = rfprevious_year[rfprevious_year['Номер ЛД'] == student_id]
        
        mae, _ = time_prediction(student_data=student_data)
        error_list.append(mae)
    except ValueError:
        continue

print('MAE total:', round(sum(error_list) / len(error_list), 2))

MAE total: 0.11
CPU times: user 35 s, sys: 406 ms, total: 35.4 s
Wall time: 35.8 s


Ошибка получилась небольшая, но это только внутри студента, сама реализация является запутанной
и тяжело настраиваемой. 

Поэтому я решил реализовать более понятную функцию, воспользовавшись только sklearn
В этой реализации я использовал LinearRegression.

<font size=4>Предсказание индекса успеваемости на основе временного ряда. sklearn</font>

In [150]:
def predict_performance(student_data: pd.DataFrame, 
                        test_data: pd.DataFrame = None):
    """
    Функция для предсказания индекса успеваемости.
    Она способна делать предсказания и находить среднюю абсолютную
    ошибку, если передан проверяющий датасет.
    :param student_data: Датасет со студентами;
    :param test_data: Датасет для проверки предсказаний
    :return: 
    """
    X = np.array((student_data['period'] - student_data['period'].min()).dt.days).reshape(-1, 1)
    y = student_data['performance_index'].values

    # Обучение модели
    model = LinearRegression()
    model.fit(X, y)

    next_year = student_data['period'].max() + pd.DateOffset(months=6)
    X_next = np.array((next_year - student_data['period'].min()).days).reshape(-1, 1)
    
    performance_index_pred = model.predict(X_next)
    mae = None
    
    if test_data is not None:
        y_next = test_data[test_data['period'] == next_year]['performance_index'].values
        mae = mean_absolute_error(performance_index_pred, y_next)
        
    performance_index_pred = round(performance_index_pred[0], 2)

    return performance_index_pred, mae, next_year

In [151]:
pred_data = rfprevious_year[rfprevious_year['period'] < pd.to_datetime('2023-06-01')]
test_data = rfprevious_year[rfprevious_year['period'] == pd.to_datetime('2023-06-01')]

counter = 0
predictions = []
for student_id in test_data['Номер ЛД'].unique():
    try:
        predict_data = pred_data[pred_data['Номер ЛД'] == student_id]
        validate_data = test_data[test_data['Номер ЛД'] == student_id]
        pred, mae, _ = predict_performance(predict_data, validate_data)
        predictions.append({'student_id': student_id, 'predicted_performance_index': pred, 'MAE': mae})
        counter += 1
    except KeyError:
        continue
    except ValueError:
        continue

predictions_df = pd.DataFrame(predictions)
print('MAE: ', round(predictions_df['MAE'].mean(), 2))
print(f'Обработано студентов: {counter}/{test_data['Номер ЛД'].nunique()}')

MAE:  0.12
Обработано студентов: 6095/6171


Модель показала себя хорошо и получилось создать удобную функцию для обучения и предсказания
Поэтому попробуем взять рандомных студентов и получить для них предсказания на 2024 год

In [152]:
def get_predict(predict_data: pd.DataFrame):
    predictions = []
    for student_id in predict_data['Номер ЛД'].unique():
        student_data = predict_data[predict_data['Номер ЛД'] == student_id]
        pred, _, next_year = predict_performance(student_data)
        predictions.append({'student_id': student_id, 
                            'predicted_performance_index': pred,
                            'period': next_year})
        
    predictions_df = pd.DataFrame(predictions)
    return predictions_df

In [153]:
np.random.seed(5)

merge_index = fprevious_year.index.names

# соединяем семестры прошлых годов обучения (до 2023-2024) и текущего (2023-2024 1 полугодие)
# На 2023-2024 2 полугодие будем делать предсказания
total_predict_data = pd.merge(fprevious_year, ftest_current_year, on=merge_index, how='outer')
total_predict_data = (total_predict_data['performance_index_x']
                      .combine_first(total_predict_data['performance_index_y'])
                      .reset_index()
                      .rename(columns={'performance_index_x': 'performance_index'}))

# Выбираем 3 студентов
random_students_id = np.random.choice(predict_current_year['Номер ЛД'].unique(), 3)
# Находим студентов среди всей истории семестров
random_students_data = total_predict_data[total_predict_data['Номер ЛД'].isin(random_students_id)]
# Делаем предсказания для 3 студентов
pred_performance = get_predict(random_students_data).rename(columns={'student_id': 'Номер ЛД'})

In [154]:
# Обрабатываем датафрейм, чтобы соединить фактические значения и предсказанные
students = (random_students_data
            .drop(['Уровень подготовки', 'Учебная группа', 
                   'Специальность/направление'], axis=1)
            .assign(real=1)
            .merge(pred_performance, on=['Номер ЛД', 'period'], how='outer')
            .assign(performance_index=lambda df: df['performance_index']
                    .combine_first(df['predicted_performance_index']))
            .drop('predicted_performance_index', axis=1)
            .fillna(0))
students.head(10)

,Номер ЛД,period,performance_index,real
0,2104406,2021-12-01,0.866667,1.0
1,2104406,2022-06-01,0.825000,1.0
2,2104406,2022-12-01,0.742857,1.0
3,2104406,2023-06-01,0.840000,1.0
4,2104406,2023-12-01,0.066667,1.0
5,2104406,2024-06-01,0.190000,0.0
6,2110260,2021-12-01,1.000000,1.0
7,2110260,2022-06-01,1.000000,1.0
8,2110260,2022-12-01,1.000000,1.0
9,2110260,2023-06-01,1.000000,1.0


In [155]:
performance_plot(sample_students=students, max_students=3, predicted=True)

<font size=4>Кластеризация студентов. KMeans</font>

Следующей идеей была кластеризация студентов - возможность разбирать студентов на группы и провести анализ.
Такой способ может помочь определить студентов, которым стоит уделить внимание. 
Например, советовать курсы двоечникам и олимпиады отличникам.

Для примера я взял Бакалавров по специальности "Информатика и вычислительная техника"

In [156]:
bachelor_data = (rfprevious_year
                 .query('`Уровень подготовки` == "Бакалавр" & '
                         '`Специальность/направление` == "Информатика и вычислительная техника"')
                 .drop(['Уровень подготовки', 'Учебная группа', 'Специальность/направление'], axis=1)
                 .groupby(['Номер ЛД'])
                 .agg({'performance_index': 'mean', 'repeat_num': 'mean'})
                 .reset_index())
bachelor_data.head()

,Номер ЛД,performance_index,repeat_num
0,1603860,0.612169,0.333333
1,1606068,0.503704,0.000000
2,1700282,0.609286,0.166667
3,1801366,0.617183,0.166667
4,1809323,0.641772,0.333333


In [157]:
# Нормализация данных
scaler = StandardScaler()
scaled_features = scaler.fit_transform(bachelor_data[['performance_index', 'repeat_num']])

kmeans = KMeans(n_clusters=4, random_state=42)
bachelor_data['cluster'] = kmeans.fit_predict(scaled_features)
bachelor_data['cluster'] = bachelor_data['cluster'].astype(str)

fig = px.scatter(
    bachelor_data,
    x='performance_index',
    y='repeat_num',
    title='Кластеризация студентов по индексу успеваемости и количеству пересдач',
    labels={'performance_index': 'Индекс успеваемости', 
            'repeat_num': 'Количество пересдач'},
    color='cluster',
    hover_data='Номер ЛД',
)

fig.update_layout(legend_title_text='Кластер')
fig.show()

In [158]:
cluster_stats = (bachelor_data
                 .groupby('cluster')
                 .agg({'performance_index': ['mean', 'std'],
                       'repeat_num': ['mean', 'std'],})
                 .reset_index())
cluster_stats

cluster performance_index           repeat_num          
                       mean       std       mean       std
0       0          0.884676  0.076267   0.026974  0.073295
1       1          0.785272  0.070414   0.441761  0.155716
2       2          0.638780  0.135115   1.816667  0.529675
3       3          0.554389  0.094262   0.286950  0.232618

Самым оптимальным решением было установить n_cluster = 4.
Таким образом, мы смогли разбить студентов на группы.
0 группа - относится к студентам со средним индексом 0.88 - отличники 
1 группа - относится к студентам со средним индексом 0.78 - хорошисты
2 группа - относится к студентам со средним индексом 0.63 - между троечниками и хорошистами
3 группа - относится к студентам со средним индексом 0.55 - между троечниками и двоечниками